### Objectives

First, create a graph representation of the information we have. 
This would be information about the elements, and the nodes. 
Using this, we will feed it into our SVM, or a Random Forest, and see from there onwards

In [263]:
from Generate_Excel import *

In [223]:
import pandas as pd
import igraph as ig

def Mitograph_Graph(FileName):
    # Read information from the gnet file, and create a graph object
    # generate a function that takes in the filename and gives out the graph
    G = pd.read_table(FileName + '.gnet', skiprows=1, sep='\t', names=['Source', 'Target', 'Length'])
    graph = ig.Graph.TupleList(G.itertuples(index=False), directed=False, edge_attrs=['Length'])
    layout = graph.layout_auto()  # Automatic layout calculation
    graph.vs['degree'] = graph.degree()
    # ig.plot(graph, layout=layout)
    Vol = pd.read_table(FileName + '.cc', skiprows=0, sep='\t')
    Vol.columns = ['node', 'cc', 'vol_cc']

    ids = [int(node['name']) for node in graph.vs]
    graph.vs['cc_vol'] = [Vol.loc[Vol['node'] == node, 'vol_cc'].values[0] for node in ids]
    graph.vs['cc'] = [Vol.loc[Vol['node'] == node, 'cc'].values[0] for node in ids]
    return graph, Vol

In [314]:
df_dir = '/home/mitosim2/Desktop/BIRATAL/Tcdd11s_subsetted/AnalysisFiles_TCDD1nM4hr1_Subset/structural/TCDD1nM4hr1_Subset_structural'
df_dir_2 = '/home/mitosim2/Desktop/BIRATAL/Tcdd11s_subsetted/AnalysisFiles_TCDD1nM4hr1_Subset/full_Table_TCDD1nM4hr1_Subset.csv' 
df = pd.read_csv(df_dir_2)
graph2, Vol = Mitograph_Graph(df_dir)

In [237]:
FileName = df_dir
G = pd.read_table(FileName + '.gnet', skiprows=1, sep='\t', names=['Source', 'Target', 'Length'])
# df = df[['line_id', 'node', 'cc_x']]

In [246]:


# Group by 'line_id' and filter out groups with no nodes
result = df.groupby('line_id').filter(lambda group: (len(group['node'].dropna()) == 0))

# The 'result' DataFrame will now contain only the line_ids with no nodes
result.line_id.unique()


array([130, 234, 166, 107, 193, 200,  88, 270, 116])

In [252]:
df[df['node'] == 130][['cc_x', 'line_id', 'node']]

,cc_x,line_id,node
1496,48,87,130.0


In [259]:
df[df['node'] == 130][['cc_x', 'line_id', 'node']]+

,cc_x,line_id,node
1496,48,87,130.0


In [261]:
G[G['Target'] == 0]

,Source,Target,Length
269,1,0,0.208


In [221]:
# in df, find the rows which have the same source and target as the rows in G
s, t, l = G.values[39]

df.loc[(df['node'] == s) ^ (df['node'] == t)]


,line_id,node,cc_x
34,277,227.0,0
235,128,227.0,19
285,277,227.0,19
297,39,227.0,19
299,39,325.0,19
302,40,325.0,19
307,41,325.0,19


In [267]:
a, p, c, n, f = Generate_Mito_Tables('/home/mitosim2/Desktop/BIRATAL/Tcdd11s_subsetted/')

1 folders found. Now running...
csv File generated


In [368]:
groupn = n[0].groupby('line_id')
dataframelist = [group for _, group in groupn]

In [369]:
for i in dataframelist: 
    i['node_2'] = i['node'].shift(-1)
    i.dropna(subset=['node_2'], inplace=True)

In [370]:
eledf = df[['line_id', 'element_Volume_Voxel', 'element_pixel_intensity_405', 'element_length_(um)', 'element_pixel_intensity_488']].drop_duplicates(subset=['line_id'])

In [374]:
eledf

,line_id,element_Volume_Voxel,element_pixel_intensity_405,element_length_(um),element_pixel_intensity_488
0,130,0.244530,48.571429,1.067511,497.428571
7,210,0.087399,48.000000,0.441235,499.312500
11,51,0.904549,30.065217,3.049798,633.663043
34,277,0.386305,26.833333,1.772482,449.291667
46,126,0.217493,37.416667,0.806042,634.333333
...,...,...,...,...,...
2326,42,0.151444,24.136364,1.092224,479.545455
2337,38,0.056781,25.937500,0.330596,264.875000
2341,13,0.776627,36.700000,3.175788,594.216667
2371,2,0.356443,53.272727,1.137652,757.204545


In [375]:
bigDf = pd.concat(dataframelist)
bigDf

,x,y,z,node,line_id,point_id,degree,cc,vol_cc,avg_PK_Of_element,element_connectivity,folder_name,node_2
282,9.0480,12.8960,2.5000,182,0,3,1,95,0.20550,1.0,1.000000,TCDD1nM4hr1_Subset,367.0
561,8.4240,1.8720,1.5000,365,1,9,1,104,0.15683,1.0,1.000000,TCDD1nM4hr1_Subset,366.0
559,8.0080,3.6400,1.5000,363,2,10,1,103,0.24336,1.0,1.000000,TCDD1nM4hr1_Subset,364.0
473,7.1760,5.4080,2.0000,300,3,13,1,98,0.51917,1.0,1.000000,TCDD1nM4hr1_Subset,362.0
462,7.8693,7.9733,2.0000,293,4,2,3,29,5.49994,2.0,0.500000,TCDD1nM4hr1_Subset,361.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,3.5707,25.8613,1.8333,227,277,0,3,19,12.67635,4.0,0.250000,TCDD1nM4hr1_Subset,228.0
99,9.1867,29.6747,3.5000,80,278,3,3,22,10.12918,3.0,0.333333,TCDD1nM4hr1_Subset,154.0
38,2.0800,20.3840,6.0000,38,279,12,1,26,1.34659,2.0,0.500000,TCDD1nM4hr1_Subset,65.0
24,7.9040,13.9360,7.0000,24,280,19,1,17,0.69763,1.0,1.000000,TCDD1nM4hr1_Subset,42.0


In [378]:
bigDf.merge(eledf, on='line_id')
bigDf

,x,y,z,node,line_id,point_id,degree,cc,vol_cc,avg_PK_Of_element,element_connectivity,folder_name,node_2
282,9.0480,12.8960,2.5000,182,0,3,1,95,0.20550,1.0,1.000000,TCDD1nM4hr1_Subset,367.0
561,8.4240,1.8720,1.5000,365,1,9,1,104,0.15683,1.0,1.000000,TCDD1nM4hr1_Subset,366.0
559,8.0080,3.6400,1.5000,363,2,10,1,103,0.24336,1.0,1.000000,TCDD1nM4hr1_Subset,364.0
473,7.1760,5.4080,2.0000,300,3,13,1,98,0.51917,1.0,1.000000,TCDD1nM4hr1_Subset,362.0
462,7.8693,7.9733,2.0000,293,4,2,3,29,5.49994,2.0,0.500000,TCDD1nM4hr1_Subset,361.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,3.5707,25.8613,1.8333,227,277,0,3,19,12.67635,4.0,0.250000,TCDD1nM4hr1_Subset,228.0
99,9.1867,29.6747,3.5000,80,278,3,3,22,10.12918,3.0,0.333333,TCDD1nM4hr1_Subset,154.0
38,2.0800,20.3840,6.0000,38,279,12,1,26,1.34659,2.0,0.500000,TCDD1nM4hr1_Subset,65.0
24,7.9040,13.9360,7.0000,24,280,19,1,17,0.69763,1.0,1.000000,TCDD1nM4hr1_Subset,42.0


In [ ]:
bigDf = bigDf[['node', 'node_2', 'line_id', 'element_pixel_intensity_ratio', 'element_pixel_intensity_405', 'element_length_(um)', 'element_pixel_intensity_488']]

In [349]:
graph = ig.Graph.TupleList(bigDf[['node', 'node_2', 'line_id', 'element_pixel_intensity_ratio']].itertuples(index=False), directed=False, edge_attrs=['line_id'])

# ids = [int(node['name']) for node in graph.vs]
# graph.vs['cc_vol'] = [df.loc[df['node'] == node, 'degree'].values[0] for node in ids]
# graph.vs['488'] = [df.loc[df['node'] == node, 'pixel_intensity_488'].values[0] for node in ids]
# graph.vs['405'] = [df.loc[df['node'] == node, 'pixel_intensity_405'].values[0] for node in ids]
# graph.vs['ratio'] = [df.loc[df['node'] == node, 'pixel_intensity_ratio'].values[0] for node in ids]
# graph.vs['structural'] = [df.loc[df['node'] == node, 'pixel_intensity_555'].values[0] for node in ids]

In [331]:
ids = [int(id['line_id']) for id in graph.es]
# graph.es['555'] = [df.loc[df['node'] == node, 'element_pixel_intensity_555'].values[0] for node in ids]
ids

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [332]:
graph.es['line_id']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,